In [1]:
import os
import pandas as pd
import json
import glob

In [ ]:
download_folder = '/content/downloaded_files'
os.makedirs(download_folder, exist_ok=True)

with open('/content/50 - Copy.txt', 'r') as f:
    categories = [line.strip() for line in f]

In [ ]:
len(categories)

65

In [ ]:
for category in categories:

    download_path = os.path.join(download_folder, category)
    print(f"Downloading {category}...")
    !gsutil cp gs://quickdraw_dataset/full/simplified/{category} {download_path}

print("All files have been downloaded successfully.")

Copying gs://quickdraw_dataset/full/simplified/apple.ndjson...
\
Operation completed over 1 objects/56.1 MiB.                                     
Copying gs://quickdraw_dataset/full/simplified/banana.ndjson...
- [1 files][104.8 MiB/104.8 MiB]                                                
Operation completed over 1 objects/104.8 MiB.                                    
Copying gs://quickdraw_dataset/full/simplified/bear.ndjson...
- [1 files][ 77.9 MiB/ 77.9 MiB]                                                
Operation completed over 1 objects/77.9 MiB.                                     
Copying gs://quickdraw_dataset/full/simplified/bird.ndjson...
- [1 files][ 65.9 MiB/ 65.9 MiB]                                                
Operation completed over 1 objects/65.9 MiB.                                     
CommandException: No URLs matched: gs://quickdraw_dataset/full/simplified/birthday
Copying gs://quickdraw_dataset/full/simplified/bus.ndjson...
\
Operation completed over 1 obj

In [ ]:
# [!gsutil cp gs://quickdraw_dataset/full/simplified/{category} {os.path.join(download_folder, category)} for category in categories]

In [ ]:
import os
import dask.dataframe as dd

# تحديد مسار المجلد الذي يحتوي على الملفات
download_folder = '/content/downloaded_files'

# الحصول على قائمة بجميع الملفات التي تنتهي بـ .ndjson
file_list = [
    os.path.join(download_folder, file)
    for file in os.listdir(download_folder)
    if file.endswith('.ndjson')
]

# معالجة كل ملف
for file_path in file_list:
    # قراءة الملف باستخدام Dask
    ddf = dd.read_json(file_path, lines=True)

    # أخذ عينة بنسبة 1%
    sample_ddf = ddf.sample(frac=0.01, random_state=42)

    # تحويل العينة إلى Pandas DataFrame وحفظها كملف CSV
    file_name = os.path.basename(file_path).replace('.ndjson', '_sample.csv')
    output_path = os.path.join('/content/sample_data', file_name)
    sample_ddf.compute().to_csv(output_path, index=False)

    print(f"Saved sample for {file_name}")

print("All files have been processed successfully.")

Saved sample for face_sample.csv
Saved sample for sun_sample.csv
Saved sample for bear_sample.csv
Saved sample for bus_sample.csv
Saved sample for mouse_sample.csv
Saved sample for rainbow_sample.csv
Saved sample for potato_sample.csv
Saved sample for butterfly_sample.csv
Saved sample for tree_sample.csv
Saved sample for car_sample.csv
Saved sample for television_sample.csv
Saved sample for dog_sample.csv
Saved sample for frog_sample.csv
Saved sample for nose_sample.csv
Saved sample for circle_sample.csv
Saved sample for banana_sample.csv
Saved sample for snake_sample.csv
Saved sample for cat_sample.csv
Saved sample for door_sample.csv
Saved sample for pizza_sample.csv
Saved sample for carrot_sample.csv
Saved sample for cloud_sample.csv
Saved sample for eye_sample.csv
Saved sample for lion_sample.csv
Saved sample for camera_sample.csv
Saved sample for leaf_sample.csv
Saved sample for cow_sample.csv
Saved sample for hat_sample.csv
Saved sample for apple_sample.csv
Saved sample for snowm

In [5]:
import os
import pandas as pd

csv_folder = '/content/'

csv_files = [
    os.path.join(csv_folder, file)
    for file in os.listdir(csv_folder)
    if file.endswith('.csv')
]

data_df = pd.concat(
    [pd.read_csv(file) for file in csv_files],
    ignore_index=True
)


In [6]:
len(data_df)

76137

In [7]:
len(csv_files)

50

In [30]:
import pandas as pd

def process_drawing(drawing):
    points = []
    for stroke_id, stroke in enumerate(drawing):
        # if len(stroke) != 2 or not all(isinstance(coord, list) for coord in stroke):
            # print(f"Skipping invalid stroke at stroke_id={stroke_id}: {stroke}")
            # continue

        x_coords, y_coords = stroke
        # if len(x_coords) != len(y_coords):
        #     # print(f"Skipping mismatched stroke at stroke_id={stroke_id}: x_coords={x_coords}, y_coords={y_coords}")
        #     continue

        for x, y in zip(x_coords, y_coords):
            points.append((x, y, stroke_id))
    return points



In [33]:
processed_data = []
for _, row in data_df.iterrows():
  drawing = row['drawing']
  points = process_drawing(drawing)
  processed_data.extend(points)

processed_df = pd.DataFrame(processed_data, columns=['x', 'y', 's'])

# حفظ البيانات المحولة إلى ملف CSV
output_path = '/content/processed_cat_data.csv'
processed_df.to_csv(output_path, index=False)

print(f"Processed data saved to {output_path}")

ValueError: not enough values to unpack (expected 2, got 1)

In [43]:
import pandas as pd
import ast  # لمعالجة النصوص الآمنة

# دالة لمعالجة كل رسم
def process_drawing(drawing):
    points = []
    for stroke_id, stroke in enumerate(drawing):
        # التحقق من صحة الشطب (stroke)
        if not isinstance(stroke, list) or len(stroke) != 2:
            continue

        x_coords, y_coords = stroke
        # التحقق من أن الإحداثيات تحتوي على نفس الطول
        if len(x_coords) != len(y_coords):
            continue

        # إضافة النقاط مع معرف الشطب
        points.extend([(x, y, stroke_id) for x, y in zip(x_coords, y_coords)])

    return points

# قراءة البيانات الأصلية
# افترض أن البيانات موجودة في عمود 'drawing' داخل data_df
# يمكنك تعديل هذا الجزء حسب مصدر البيانات الخاص بك
try:
    processed_data = [
        point
        for _, row in data_df.iterrows()
        for point in process_drawing(ast.literal_eval(row['drawing']))
    ]
except Exception as e:
    print(f"Error processing data: {e}")
    processed_data = []

# تحويل البيانات إلى DataFrame
processed_df = pd.DataFrame(processed_data, columns=['x', 'y', 's'])

# حفظ البيانات المحولة إلى ملف CSV
output_path = '/content/processed_cat_data.csv'
processed_df.to_csv(output_path, index=False)

print(f"Processed data saved to {output_path}")

Processed data saved to /content/processed_cat_data.csv


In [44]:
processed_df

,x,y,s
0,61,90,0
1,69,79,0
2,85,42,0
3,111,17,0
4,173,0,0
...,...,...,...
3688891,251,63,3
3688892,255,40,3
3688893,8,98,4
3688894,20,63,4


In [45]:
len(processed_df)

3688896

In [32]:
processed_data = []
for row in data_df['drawing']:
  drawing = row
  points = process_drawing(drawing)
  processed_data.extend(points)

processed_df = pd.DataFrame(processed_data, columns=['x', 'y', 's'])

# حفظ البيانات المحولة إلى ملف CSV
output_path = '/content/processed_cat_data.csv'
processed_df.to_csv(output_path, index=False)

print(f"Processed data saved to {output_path}")

ValueError: not enough values to unpack (expected 2, got 1)

In [24]:
processed_df

,x,y,s


In [15]:
points

[]

In [23]:
import ast  # لإضافة دعم لتقييم النصوص الآمنة

processed_data = []
for _, row in data_df.iterrows():
    # تحويل النص إلى قائمة باستخدام ast.literal_eval
    try:
        drawing = ast.literal_eval(row['drawing'])  # تحويل النص إلى قائمة
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing drawing data: {e}")
        continue

    points = process_drawing(drawing)
    processed_data.extend(points)